In [38]:
!pip install folium

     -------------------------------------- 102.3/102.3 kB 3.0 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns
import plotly.express as px
import folium

C:\Users\phi49\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## 서울 지하철 내 장애인 편의시설 분포

### 1. 구 별 장애인 수 분포

In [3]:
# 데이터 불러오기
df1 = pd.read_csv('../Data Collection/구별 장애인 수.csv', encoding='utf-8-sig')

# 데이터 필요한 부분만 정리
df1 = df1.loc[3:,['자치구', '합계']]
df1.reset_index(drop=True, inplace=True)

for i, data in enumerate(df1['합계']):
    df1['합계'][i] = data.replace(',', '')

df1['합계'] = df1['합계'].astype('int')

df1.index = df1['자치구']
del df1['자치구']

In [4]:
# Folium 시각화
m = folium.Map(location=[37.562225, 126.978555], tiles="OpenStreetMap", zoom_start=11)

state_geo = 'https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json'

folium.Choropleth(
    geo_data = state_geo,
    data = df1['합계'],
    key_on = 'feature.properties.name',
    fill_color = 'BuPu',
    # fill_opacity=0.7,
    # line_opacity=0.3,
    bins = [5000,10000,15000,20000,25000,30000],
).add_to(m)
m.save('./images/disabled_count.html')

### 2. 지하철 내 장애인 편의시설 분포 (전체)

In [10]:
# 데이터 불러오기
df = pd.read_csv('../Data Collection/지하철 장애인 편의시설_전처리.csv', encoding='utf-8-sig')
df.head()

,구,역명,승강기 개수,에스컬레이터 개수,휠체어리프트 개수,전동휠체어 급속충전기 개수,경도,위도
0,중구,서울역(1),4,5,1,0,126.972559,37.554648
1,중구,시청(1),3,3,0,0,126.977108,37.564718
2,종로구,종각,4,2,0,1,126.982923,37.570161
3,종로구,종로3가(1),3,4,0,1,126.991806,37.571607
4,종로구,종로5가,3,0,0,0,127.001849,37.570926


In [11]:
# 결측치 제거
df = df.dropna(how='all')
df = df.dropna(how='all', axis=1)

In [12]:
# 편의시설 별 갯수 (뒤에서도 활용)
df['에스컬레이터 개수'] = df['에스컬레이터 개수'].astype('int')
df['휠체어리프트 개수'] = df['휠체어리프트 개수'].astype('int')
df['전동휠체어 급속충전기 개수'] = df['전동휠체어 급속충전기 개수'].astype('int')
df['승강기 개수'] = df['승강기 개수'].astype('int')

In [13]:
# 구별로 그룹화
group_df = df.groupby(df['구'])
total_df = group_df.sum()
del total_df['경도']

In [14]:
# 편의시설 총 갯수
total_df.sort_index(ascending=True,inplace=True)
total_df['총합'] = total_df['승강기 개수']+total_df['에스컬레이터 개수']+total_df['휠체어리프트 개수']+total_df['전동휠체어 급속충전기 개수']
total_df

,승강기 개수,에스컬레이터 개수,휠체어리프트 개수,전동휠체어 급속충전기 개수,총합
구,,,,,
강남구,69,211,5,3,288
강동구,43,80,8,4,135
강북구,8,12,0,2,22
강서구,24,38,3,2,67
관악구,14,32,0,0,46
광진구,27,46,0,6,79
구로구,20,53,16,3,92
금천구,4,12,4,0,20
노원구,33,66,5,7,111


In [16]:
# 지하철 편의시설 전체 시각화
state_geo = 'https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json'

m = folium.Map(location=[37.562225, 126.978555], tiles="OpenStreetMap", zoom_start=11)

folium.Choropleth(
    geo_data = state_geo,
    name = '지하철 내 편의시설 수',
    data = total_df['총합'],
    # columns = [df.index, df['총합']],
    key_on = 'feature.properties.name',
    fill_color = 'PuRd',
    fill_opacity=0.7,
    line_opacity=0.3,
    bins = [0,70,140,210,280,350],
    color='gray',
).add_to(m)

m.save('./images/subway_total.html')

### 3. 구 별 휠체어 리프트 분포

In [19]:
state_geo = 'https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json'

m = folium.Map(location=[37.562225, 126.978555], tiles="OpenStreetMap", zoom_start=11)

folium.Choropleth(
    geo_data = state_geo,
    name = '휠체어리프트 개수',
    data = total_df['휠체어리프트 개수'],
    columns = [df.index, df['휠체어리프트 개수']],
    key_on = 'feature.properties.name',
    fill_color = 'Purples',
    fill_opacity=0.7,
    line_opacity=0.3,
    bins = [0,5,10,15,20,25],
    color='gray',
).add_to(m)

m.save('./images/subway_wheelchairlift.html')

### 4. 구 별 휠체어 급속충전기 분포

In [20]:
state_geo = 'https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json'

m = folium.Map(location=[37.562225, 126.978555], tiles="OpenStreetMap", zoom_start=11)

folium.Choropleth(
    geo_data = state_geo,
    name = '전동휠체어 급속충전기 개수',
    data = total_df['전동휠체어 급속충전기 개수'],
    columns = [df.index, df['전동휠체어 급속충전기 개수']],
    key_on = 'feature.properties.name',
    fill_color = "PuBu",
    fill_opacity=0.7,
    line_opacity=0.3,
    bins = [0,5,10,15,20,25],
    color='gray',
).add_to(m)

m.save('./images/subway_wheelchaircharger.html')

### 5. 구 별 승강기 분포

In [21]:
state_geo = 'https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json'

m = folium.Map(location=[37.562225, 126.978555], tiles="OpenStreetMap", zoom_start=11)

folium.Choropleth(
    geo_data = state_geo,
    name = '승강기 개수',
    data = total_df['승강기 개수'],
    columns = [df.index, df['승강기 개수']],
    key_on = 'feature.properties.name',
    fill_color = 'Blues',
    fill_opacity=0.7,
    line_opacity=0.3,
    bins = [0,18,36,54,72,90],
    color='gray',
).add_to(m)

m.save('./images/subway_lift.html')